# 평가(Evaluation)

## 정확도(Accuracy)

### Titanic 엉터리분류기(대표적으로 왜곡하는 사례)

In [1]:
import numpy as np
from sklearn.base import BaseEstimator

In [2]:
class MyDummyClassifier(BaseEstimator):
    # fit(), predict() method만 재정의(overriding)
    def fit(self, X, y):        # 클래스메소드라 앞에 꼭self필요! 형식맞춰서 써줌
        pass                    #BaseEstimator를 사용하기위해서 그냥 정의만함; 학습안함
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))  # [0]행수
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 0: #여성인 경우
                pred[i, 0] = 1  #pred값을 1을 내뱉겠닼ㅋ
        return pred

In [3]:
import pandas as pd
titanic_df = pd.read_csv('../00.data/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

In [4]:
from sklearn.model_selection import train_test_split #널값을제외함;
X_train, X_test, y_train, y_test=train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

In [6]:
my_clf = MyDummyClassifier()
my_clf.fit(X_train, y_train)

In [7]:
my_pred = my_clf.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, my_pred)
#중간에 널값이있어서그런가 65%나옴;ㅁ;

0.659217877094972

## 전처리를 하고 

In [9]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수 ;ㅁ;...
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거(빅데이터에서는 필요하겠지만; 머신러닝에는필요없음)
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [15]:
import pandas as pd
titanic_df = pd.read_csv('../00.data/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df) # 이것추가??????

In [16]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test=train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

In [17]:
my_clf = MyDummyClassifier()
my_clf.fit(X_train, y_train)

In [18]:
my_pred = my_clf.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, my_pred)

0.8324022346368715

## MyFakeClassifier

### Mnist 손글씨 "Is it seven"  구글링

In [21]:
class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass

    # 입력값으로 들어오는 X데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [27]:
from sklearn.datasets import load_digits

# 사이키런의 내장 데이터 셋인 load_digits()를 이용하여 MNIST 데이터 로딩
digits = load_digits()

#digits번호가 7번이면 True이고 astype(int)로 1로 변환
#0~9까지들어있는데 값이 7이면 True1, 아니면False0
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [31]:
# 뷸균형한 레이블 데이터 분포도 확인.
print('레이블 테스트 세트 크기:', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


In [32]:
# Fake Classifire로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
score = accuracy_score(y_test, fakepred)
print(f'모든 예측을 0으로 하여도 정확도는: {score:.3f}')

모든 예측을 0으로 하여도 정확도는: 0.900


## 오차 행렬(Confusion Matrix)

### MyFakeClassifier 사례 (ppt.7p.)

In [33]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)


array([[405,   0],
       [ 45,   0]], dtype=int64)

### 정밀도(Precision) 재현율(Recall)

In [35]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print(f'정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}')

In [36]:
import pandas as pd
titanic_df = pd.read_csv('../00.data/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

In [37]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차행렬
[[104  14]
 [ 13  48]]
정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869
